In [ ]:
# Install required packages for model training

In [ ]:
!pip install xgboost

In [ ]:
# Required imports
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, mean_absolute_percentage_error
from sklearn.metrics import classification_report

from sklearn.multiclass import OneVsOneClassifier

import copy

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

import matplotlib.pyplot as plt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
filename = '/final_key_points.csv'

TARGET_NAMES_1 = ['Neutral', 'Relaxed', 'Angry & Scared']
TARGET_NAMES_2 = ['Alert', 'Neutral_Relaxed', 'Anger_Fear_Anxious']
TARGET_NAMES_3 = ['Neutral', 'Relaxed', 'Anger_Fear_Anxious']


# read df
CSV_OUTPATH = '/content/drive/MyDrive/Coin Project 1/Development_2'
df = pd.read_csv(CSV_OUTPATH + filename, dtype=int, index_col=0)
print(len(df))

In [ ]:
'''
0 = Alert
1 = Neutral
2 = Relaxed
3 = Anger
4 = Fear
5 = Anxious
'''

def show_number_of_samples(df):
  print('Number of Samples for Class 0: ', len(df[df['emotions'] == 0]))
  print('Number of Samples for Class 1: ', len(df[df['emotions'] == 1]))
  print('Number of Samples for Class 2: ', len(df[df['emotions'] == 2]))
  print('Number of Samples for Class 3: ', len(df[df['emotions'] == 3]))
  print('Number of Samples for Class 4: ', len(df[df['emotions'] == 4]))
  print('Number of Samples for Class 5: ', len(df[df['emotions'] == 5]))
  print('---------------------------------------')

In [ ]:
# Emotions: Neutral, Relaxed, Anger_Fear
df_merged_1 = copy.deepcopy(df)

# Merge Anger and Fear to Anger
df_merged_1.loc[df['emotions'] == 4, 'emotions'] = 3 # update where emotion=4, set emotion to 3

# Delete anxious
df_merged_1 = df_merged_1[df_merged_1['emotions'] != 5]

# Delete alert
df_merged_1 = df_merged_1[df_merged_1['emotions'] != 0]

In [ ]:
# Emotions: Alert, Neutral_Relaxed, Anger_Fear_Anxious
df_merged_2 = copy.deepcopy(df)

# Merge Anger and Fear to Anger
df_merged_2.loc[df['emotions'] == 4, 'emotions'] = 3 # update where emotion=4, set emotion to 3

# Merge Anger and Anxious to Anger
df_merged_2.loc[df['emotions'] == 5, 'emotions'] = 3 # update where emotion=5, set emotion to 3

# Merge Neutral and Relaxed to Neutral
df_merged_2.loc[df['emotions'] == 2, 'emotions'] = 1 # update where emotion=5, set emotion to 3

In [ ]:
# Emotions: Neutral, Relaxed, Anger_Fear
df_merged_3 = copy.deepcopy(df)

# Merge Anger and Fear to Anger
df_merged_3.loc[df['emotions'] == 4, 'emotions'] = 3 # update where emotion=4, set emotion to 3

# Merge Anger and Anxious to Anger
df_merged_3.loc[df['emotions'] == 5, 'emotions'] = 3 # update where emotion=5, set emotion to 3

# Delete alert
df_merged_3 = df_merged_3[df_merged_3['emotions'] != 0]

In [ ]:
show_number_of_samples(df)
show_number_of_samples(df_merged_1)
show_number_of_samples(df_merged_2)
show_number_of_samples(df_merged_3)

In [ ]:
def normalize_df(df):
  lengths = []
  for i in range(6):
    lengths.append(len(df[df['emotions'] == int(i)]))
  lengths = [i for i in lengths if i != 0]
  smallest_amount = min(lengths)
  
  # reduce all values to maximum of smallest_amount
  df_return = pd.DataFrame({})
  for i in range(6):
    if len(df[df['emotions'] == int(i)]) != 0:
      df_emotion = df[df['emotions'] == int(i)].sample(smallest_amount)
      df_return = df_return.append(df_emotion)
  return df_return;

In [ ]:
df_norm_1 = normalize_df(df_merged_1)
df_norm_2 = normalize_df(df_merged_2)
df_norm_3 = normalize_df(df_merged_3)

show_number_of_samples(df_norm_1)
show_number_of_samples(df_norm_2)
show_number_of_samples(df_norm_3)

In [ ]:
def grid_search_cv(X_train, X_test, y_train, y_test, model, isOVO):
  
  learning_rate = [0.001, 0.01, 0.10, 0.20, 0.30]
  max_depth = [3, 5, 8, 10, 20]
  gamma= [0.0, 0.1, 0.2, 1]
  n_estimators = [10, 50, 100, 250]
  reg_alpha = [0.05, 0.1, 0.5, 1, 5]


  hyperparameter_grid = {'learning_rate': learning_rate,
                          'max_depth': max_depth,
                          'gamma': [0.0, 0.1, 0.2, 0.5, 1],
                          'colsample_bytree': [0.3, 0.7, 1.0],
                          'reg_alpha': reg_alpha,
                          'n_estimators': n_estimators,
                          }
                          

  columns = ['Nose_x','Nose_y','L_Eye_x','L_Eye_y','R_Eye_x','R_Eye_y','L_Ear_x','L_Ear_y','R_Ear_x','R_Ear_y','Throat_x','Throat_y','Withers_x','Withers_y','TailSet_x','TailSet_y','L_F_Paw_x','L_F_Paw_y','R_F_Paw_x','R_F_Paw_y','L_F_Wrist_x','L_F_Wrist_y','R_F_Wrist_x','R_F_Wrist_y','L_F_Elbow_x','L_F_Elbow_y','R_F_Elbow_x','R_F_Elbow_y','L_B_Paw_x','L_B_Paw_y','R_B_Paw_x','R_B_Paw_y','L_B_Hock_x','L_B_Hock_y','R_B_Hock_x','R_B_Hock_y','L_B_Stiffle_x','L_B_Stiffle_y','R_B_Stiffle_x','R_B_Stiffle_y', 'back_middle_x', 'back_middle_y', 'tail_tip_x', 'tail_tip_y', 'lip_upper_x', 'lip_upper_y','lip_lower_x','lip_lower_y', 'ear_tip_left_x', 'ear_tip_left_y', 'ear_tip_right_x', 'ear_tip_right_y']


  grid_search_rf = GridSearchCV(estimator = xgb.XGBClassifier(random_state=42), param_grid = hyperparameter_grid, cv = 4, n_jobs = -1, verbose = 1, scoring="accuracy")
  # Fit it

  if isOVO:
    ovo = OneVsOneClassifier(grid_search_rf)
    # Fit it
    ovo.fit(X_train, y_train)
    # Predict the train data
    result_train = ovo.predict(X_train)
    # Predict the test data
    result_test = ovo.predict(X_test)

    print(f'Model: {str(model)}; Accuracy for Train Set: {accuracy_score(y_train, result_train)}; Accuracy for Test Set: {accuracy_score(y_test, result_test)}')
    print(classification_report(y_test, result_test, target_names=model))
    
    # Feature Importance
    importance_sorted = sorted(zip(grid_search_rf.best_estimator_.feature_importances_, columns), reverse=True)
    print()
    print("Feature Importance: ", importance_sorted)

    rf_params = grid_search_rf.best_params_
    print()
    print("Parameter: ", rf_params)

    feature_values = [i[0] for i in importance_sorted[0:10]]
    feature_names = [i[1] for i in importance_sorted[0:10]]
    indices = list(range(0,10))

    plt.title('Feature Importances')
    plt.barh(range(len(indices)), feature_values, color='b', align='center')
    plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
    plt.xlabel('Relative Importance')
    plt.show()

  else:

    grid_search_rf.fit(X_train, y_train)
    print("BEST ", grid_search_rf.best_score_)
    # Predict the train data
    result_train = grid_search_rf.predict(X_train)
    # Predict the test data
    result_test = grid_search_rf.predict(X_test)

    print(f'Model: {str(model)}; Accuracy for Train Set: {accuracy_score(y_train, result_train)}; Accuracy for Test Set: {accuracy_score(y_test, result_test)}')
    print(classification_report(y_test, result_test, target_names=model))
    
    # Feature Importance
    importance_sorted = sorted(zip(grid_search_rf.best_estimator_.feature_importances_, columns), reverse=True)
    print()
    print("Feature Importance: ", importance_sorted)

  #   rf_params = grid_search_rf.best_estimator_.getParams(False)
    rf_params = grid_search_rf.best_params_
    print()
    print("Parameter: ", rf_params)

    feature_values = [i[0] for i in importance_sorted[0:10]]
    feature_names = [i[1] for i in importance_sorted[0:10]]
    indices = list(range(0,10))

    plt.title('Feature Importances')
    plt.barh(range(len(indices)), feature_values, color='b', align='center')
    plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
    plt.xlabel('Relative Importance')
    plt.show()

    return grid_search_rf

# XGBoost for Neutral, Relaxed, Anger_Fear (df_norm_1)

In [ ]:
# Train and Test split
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_norm_1, test_size=0.2, random_state=42)

In [ ]:
# all rows, but not the first column
X_train = train.values[:, 1:]
# all rows, but only the first column
y_train = train.values[:, :1]

X_test = test.values[:, 1:]
y_test = test.values[:, :1]

In [ ]:
y_train = y_train.reshape((1, len(y_train))).squeeze()
y_test = y_test.reshape((1, len(y_test))).squeeze()

In [ ]:
grid_search_cv(X_train, X_test, y_train, y_test, TARGET_NAMES_1, False)

In [ ]:
grid_search_cv(X_train, X_test, y_train, y_test, TARGET_NAMES_1, True)

# XGBoost for Alert, Neutral_Relaxed, Anger_Fear_Anxious (df_norm_2)

In [ ]:
# Train and Test split
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_norm_2, test_size=0.2, random_state=42)

In [ ]:
# all rows, but not the first column
X_train = train.values[:, 1:]
# all rows, but only the first column
y_train = train.values[:, :1]

X_test = test.values[:, 1:]
y_test = test.values[:, :1]

In [ ]:
y_train = y_train.reshape((1, len(y_train))).squeeze()
y_test = y_test.reshape((1, len(y_test))).squeeze()

In [ ]:
grid_search_cv(X_train, X_test, y_train, y_test, TARGET_NAMES_2, False)

In [ ]:
grid_search_cv(X_train, X_test, y_train, y_test, TARGET_NAMES_2, True)

# XGBoost for Neutral, Relaxed, Anger_Fear_Anxious (df_norm_3)

In [ ]:
# Train and Test split
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_norm_3, test_size=0.2, random_state=42)

In [ ]:
# all rows, but not the first column
X_train = train.values[:, 1:]
# all rows, but only the first column
y_train = train.values[:, :1]

X_test = test.values[:, 1:]
y_test = test.values[:, :1]

In [ ]:
y_train = y_train.reshape((1, len(y_train))).squeeze()
y_test = y_test.reshape((1, len(y_test))).squeeze()

In [ ]:
grid_search_rf = grid_search_cv(X_train, X_test, y_train, y_test, TARGET_NAMES_3, False)

In [ ]:
grid_search_cv(X_train, X_test, y_train, y_test, TARGET_NAMES_3, True)